In [1]:
!python --version

Python 3.11.7


In [2]:
# Import Libraries
import pandas as pd
import numpy as np
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
import seaborn as sns
import matplotlib.pyplot as plt


# Change setting in pandas to display all columns
pd.options.display.max_columns = None

In [3]:
# Import medical dataset
medicineDF =pd.read_csv(r"C:\Users\ashle\Desktop\MSDA WGU\Data Mining 2 -D212\dataset\medical_market_basket.csv")

In [4]:
medicineDF

,Presc01,Presc02,Presc03,Presc04,Presc05,Presc06,Presc07,Presc08,Presc09,Presc10,Presc11,Presc12,Presc13,Presc14,Presc15,Presc16,Presc17,Presc18,Presc19,Presc20
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,amlodipine,albuterol aerosol,allopurinol,pantoprazole,lorazepam,omeprazole,mometasone,fluconozole,gabapentin,pravastatin,cialis,losartan,metoprolol succinate XL,sulfamethoxazole,abilify,spironolactone,albuterol HFA,levofloxacin,promethazine,glipizide
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,citalopram,benicar,amphetamine salt combo xr,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14997,clopidogrel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14999,alprazolam,losartan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Check for columns that only contain null values.
medicineDF.isna().all()

Presc01    False
Presc02    False
Presc03    False
Presc04    False
Presc05    False
Presc06    False
Presc07    False
Presc08    False
Presc09    False
Presc10    False
Presc11    False
Presc12    False
Presc13    False
Presc14    False
Presc15    False
Presc16    False
Presc17    False
Presc18    False
Presc19    False
Presc20    False
dtype: bool

In [6]:
# Count rows that only contain null values
print(f"Rows with all null values: {medicineDF.isna().all(axis=1).sum()}")
# Drop rows that contain only null values 
medicine_NotNA = medicineDF.dropna(axis=0, how='all')
print(f"Shape of Data Frame after null rows have been removed; {medicine_NotNA.shape}")

Rows with all null values: 7501
Shape of Data Frame after null rows have been removed; (7501, 20)


In [7]:
# Convert the dataframe into a list of lists without null values.
list_of_lists = []
for i in range(len(medicine_NotNA)):
    transaction = [str(medicine_NotNA.values[i, j]) for j in range(medicine_NotNA.shape[1]) if pd.notnull(medicine_NotNA.values[i, j])]
    list_of_lists.append(transaction)


In [8]:
# Print 10 transactions
for i, transaction in enumerate(list_of_lists[0:10], start=1):
    print(f"Transaction {i}: {transaction}")

Transaction 1: ['amlodipine', 'albuterol aerosol', 'allopurinol', 'pantoprazole', 'lorazepam', 'omeprazole', 'mometasone', 'fluconozole', 'gabapentin', 'pravastatin', 'cialis', 'losartan', 'metoprolol succinate XL', 'sulfamethoxazole', 'abilify', 'spironolactone', 'albuterol HFA', 'levofloxacin', 'promethazine', 'glipizide']
Transaction 2: ['citalopram', 'benicar', 'amphetamine salt combo xr']
Transaction 3: ['enalapril']
Transaction 4: ['paroxetine', 'allopurinol']
Transaction 5: ['abilify', 'atorvastatin', 'folic acid', 'naproxen', 'losartan']
Transaction 6: ['cialis']
Transaction 7: ['hydrochlorothiazide', 'glyburide']
Transaction 8: ['metformin', 'salmeterol inhaler', 'sertraline HCI']
Transaction 9: ['metoprolol', 'carvedilol', 'losartan']
Transaction 10: ['glyburide']


In [9]:
# Encode the data using TransactionEncoder
te = TransactionEncoder()
te_array = te.fit(list_of_lists).transform(list_of_lists)
encodedDF = pd.DataFrame(te_array, columns=te.columns_)


In [10]:
encodedDF

,Duloxetine,Premarin,Yaz,abilify,acetaminophen,actonel,albuterol HFA,albuterol aerosol,alendronate,allopurinol,alprazolam,amitriptyline,amlodipine,amoxicillin,amphetamine,amphetamine salt combo,amphetamine salt combo xr,atenolol,atorvastatin,azithromycin,benazepril,benicar,boniva,bupropion sr,carisoprodol,carvedilol,cefdinir,celebrex,celecoxib,cephalexin,cialis,ciprofloxacin,citalopram,clavulanate K+,clonazepam,clonidine HCI,clopidogrel,clotrimazole,codeine,crestor,cyclobenzaprine,cymbalta,dextroamphetamine XR,diazepam,diclofenac sodium,doxycycline hyclate,enalapril,escitalopram,esomeprazole,ezetimibe,fenofibrate,fexofenadine,finasteride,flovent hfa 110mcg inhaler,fluconozole,fluoxetine HCI,fluticasone,fluticasone nasal spray,folic acid,furosemide,gabapentin,glimepiride,glipizide,glyburide,hydrochlorothiazide,hydrocodone,hydrocortisone 2.5% cream,ibuprophen,isosorbide mononitrate,lansoprazole,lantus,levofloxacin,levothyroxine sodium,lisinopril,lorazepam,losartan,lovastatin,meloxicam,metformin,metformin HCI,methylprednisone,metoprolol,metoprolol succinate XL,metoprolol tartrate,mometasone,naproxen,omeprazole,oxycodone,pantoprazole,paroxetine,pioglitazone,potassium Chloride,pravastatin,prednisone,pregabalin,promethazine,quetiapine,ranitidine,rosuvastatin,salmeterol inhaler,sertraline HCI,simvastatin,spironolactone,sulfamethoxazole,synthroid,tamsulosin,temezepam,topiramate,tramadol,trazodone HCI,triamcinolone Ace topical,triamterene,trimethoprim DS,valaciclovir,valsartan,venlafaxine XR,verapamil SR,viagra,zolpidem
0,False,False,False,True,False,False,True,True,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False,True,False,False,True,True,False,False,False,False,False,False,True,False,True,False,True,False,True,False,False,False,True,False,False,True,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,Fa

In [11]:
# Display all columns in the encoded dataframe.
list(encodedDF.columns)

['Duloxetine',
 'Premarin',
 'Yaz',
 'abilify',
 'acetaminophen',
 'actonel',
 'albuterol HFA',
 'albuterol aerosol',
 'alendronate',
 'allopurinol',
 'alprazolam',
 'amitriptyline',
 'amlodipine',
 'amoxicillin',
 'amphetamine',
 'amphetamine salt combo',
 'amphetamine salt combo xr',
 'atenolol',
 'atorvastatin',
 'azithromycin',
 'benazepril',
 'benicar',
 'boniva',
 'bupropion sr',
 'carisoprodol',
 'carvedilol',
 'cefdinir',
 'celebrex',
 'celecoxib',
 'cephalexin',
 'cialis',
 'ciprofloxacin',
 'citalopram',
 'clavulanate K+',
 'clonazepam',
 'clonidine HCI',
 'clopidogrel',
 'clotrimazole',
 'codeine',
 'crestor',
 'cyclobenzaprine',
 'cymbalta',
 'dextroamphetamine XR',
 'diazepam',
 'diclofenac sodium',
 'doxycycline hyclate',
 'enalapril',
 'escitalopram',
 'esomeprazole',
 'ezetimibe',
 'fenofibrate',
 'fexofenadine',
 'finasteride',
 'flovent hfa 110mcg inhaler',
 'fluconozole',
 'fluoxetine HCI',
 'fluticasone',
 'fluticasone nasal spray',
 'folic acid',
 'furosemide',
 'g

In [12]:
encodedDF.shape

(7501, 119)

In [13]:
# Export cleaned data
cleanedData = encodedDF
cleanedData.to_csv(r"C:\Users\ashle\Desktop\MSDA WGU\Data Mining 2 -D212\task 3\cleaned data\cleanedData.csv", index=False)

In [14]:
# Applying Apriori to the dataset to identify frequently prescribed medications.
rules = apriori(cleanedData, min_support=0.02, use_colnames=True)
rules.head()

,support,itemsets
0,0.046794,(Premarin)
1,0.238368,(abilify)
2,0.020397,(albuterol aerosol)
3,0.033329,(allopurinol)
4,0.079323,(alprazolam)


In [15]:
rules.shape

(103, 2)

In [16]:
# Creating association rules from the item sets with a filter to include only those with a lift above 1.
ruleTable = association_rules(rules, metric='lift', min_threshold =1)
ruleTable.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(amlodipine),(abilify),0.071457,0.238368,0.023597,0.330224,1.385352,1.0,0.006564,1.137144,0.299568,0.082441,0.120604,0.214609
1,(abilify),(amlodipine),0.238368,0.071457,0.023597,0.098993,1.385352,1.0,0.006564,1.030562,0.365218,0.082441,0.029655,0.214609
2,(abilify),(amphetamine salt combo),0.238368,0.068391,0.024397,0.102349,1.496530,1.0,0.008095,1.037830,0.435627,0.086402,0.036451,0.229537
3,(amphetamine salt combo),(abilify),0.068391,0.238368,0.024397,0.356725,1.496530,1.0,0.008095,1.183991,0.356144,0.086402,0.155399,0.229537
4,(abilify),(amphetamine salt combo xr),0.238368,0.179709,0.050927,0.213647,1.188845,1.0,0.008090,1.043158,0.208562,0.138707,0.041372,0.248515


In [17]:
ruleTable.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94 entries, 0 to 93
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   antecedents         94 non-null     object 
 1   consequents         94 non-null     object 
 2   antecedent support  94 non-null     float64
 3   consequent support  94 non-null     float64
 4   support             94 non-null     float64
 5   confidence          94 non-null     float64
 6   lift                94 non-null     float64
 7   representativity    94 non-null     float64
 8   leverage            94 non-null     float64
 9   conviction          94 non-null     float64
 10  zhangs_metric       94 non-null     float64
 11  jaccard             94 non-null     float64
 12  certainty           94 non-null     float64
 13  kulczynski          94 non-null     float64
dtypes: float64(12), object(2)
memory usage: 10.4+ KB


In [18]:
# Export association rules tables
ruleTable.to_csv(r"C:\Users\ashle\Desktop\MSDA WGU\Data Mining 2 -D212\task 3\cleaned data\assocation_rules.csv", float_format='%.6f')

In [19]:
# Print rules with their support, lift, and confidence values
ruleTable[['antecedents', 'consequents', 'support', 'lift', 'confidence']]

,antecedents,consequents,support,lift,confidence
0,(amlodipine),(abilify),0.023597,1.385352,0.330224
1,(abilify),(amlodipine),0.023597,1.385352,0.098993
2,(abilify),(amphetamine salt combo),0.024397,1.496530,0.102349
3,(amphetamine salt combo),(abilify),0.024397,1.496530,0.356725
4,(abilify),(amphetamine salt combo xr),0.050927,1.188845,0.213647
...,...,...,...,...,...
89,(diazepam),(metoprolol),0.022930,1.468215,0.139951
90,(glyburide),(doxycycline hyclate),0.020131,1.239135,0.117785
91,(doxycycline hyclate),(glyburide),0.020131,1.239135,0.211781
92,(glyburide),(losartan),0.028530,1.263488,0.166927


In [20]:
# Printing the three rules with the highest support value
topThreeRulesSupport = ruleTable.sort_values(by='support', ascending=False).head(3)
topThreeRulesSupport

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
8,(abilify),(carvedilol),0.238368,0.174110,0.059725,0.250559,1.439085,1.0,0.018223,1.102008,0.400606,0.169312,0.092566,0.296796
9,(carvedilol),(abilify),0.174110,0.238368,0.059725,0.343032,1.439085,1.0,0.018223,1.159314,0.369437,0.169312,0.137421,0.296796
19,(diazepam),(abilify),0.163845,0.238368,0.052660,0.321400,1.348332,1.0,0.013604,1.122357,0.308965,0.150648,0.109018,0.271158


In [21]:
# Printing the three rules with the highest confidence value
topThreeRulesConfidence = ruleTable.sort_values(by='confidence', ascending=False).head(3)
topThreeRulesConfidence

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
31,(metformin),(abilify),0.050527,0.238368,0.023064,0.456464,1.914955,1.0,0.011020,1.401255,0.503221,0.086760,0.286354,0.276610
25,(glipizide),(abilify),0.065858,0.238368,0.027596,0.419028,1.757904,1.0,0.011898,1.310962,0.461536,0.099759,0.237201,0.267400
29,(lisinopril),(abilify),0.098254,0.238368,0.040928,0.416554,1.747522,1.0,0.017507,1.305401,0.474369,0.138413,0.233952,0.294127


In [22]:
# Printing the three rules with the highest lift value
topThreeRulesLift = ruleTable.sort_values(by='lift', ascending=False).head(3)
topThreeRulesLift

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
74,(carvedilol),(lisinopril),0.174110,0.098254,0.039195,0.225115,2.291162,1.0,0.022088,1.163716,0.682343,0.168096,0.140684,0.312015
75,(lisinopril),(carvedilol),0.098254,0.174110,0.039195,0.398915,2.291162,1.0,0.022088,1.373997,0.624943,0.168096,0.272197,0.312015
73,(glipizide),(carvedilol),0.065858,0.174110,0.022930,0.348178,1.999758,1.0,0.011464,1.267048,0.535186,0.105651,0.210764,0.239939


In [23]:
# Printing the top 10 most common rules
top10Rules = ruleTable.sort_values(by=['support', 'confidence', 'lift'], ascending=False).head(10)
top10Rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
9,(carvedilol),(abilify),0.174110,0.238368,0.059725,0.343032,1.439085,1.0,0.018223,1.159314,0.369437,0.169312,0.137421,0.296796
8,(abilify),(carvedilol),0.238368,0.174110,0.059725,0.250559,1.439085,1.0,0.018223,1.102008,0.400606,0.169312,0.092566,0.296796
19,(diazepam),(abilify),0.163845,0.238368,0.052660,0.321400,1.348332,1.0,0.013604,1.122357,0.308965,0.150648,0.109018,0.271158
18,(abilify),(diazepam),0.238368,0.163845,0.052660,0.220917,1.348332,1.0,0.013604,1.073256,0.339197,0.150648,0.068256,0.271158
5,(amphetamine salt combo xr),(abilify),0.179709,0.238368,0.050927,0.283383,1.188845,1.0,0.008090,1.062815,0.193648,0.138707,0.059103,0.248515
4,(abilify),(amphetamine salt combo xr),0.238368,0.179709,0.050927,0.213647,1.188845,1.0,0.008090,1.043158,0.208562,0.138707,0.041372,0.248515
7,(atorvastatin),(abilify),0.129583,0.238368,0.047994,0.370370,1.553774,1.0,0.017105,1.209650,0.409465,0.150000,0.173315,0.285856
6,(abilify),(atorvastatin),0.238368,0.129583,0.047994,0.201342,1.553774,1.0,0.017105,1.089850,0.467950,0.150000,0.082443,0.285856
29,(lisinopril),(abilify),0.098254,0.238368,0.040928,0.416554,1.747522,1.0,0.017507,1.305401,0.474369,0.138413,0.233952,0.294127
28,(abilify),(lisinopril),0.238368,0.098254,0.040928,0.171700,1.747522,1.0,0.017507,1.088672,0.561638,0.138413,0.081449,0.294127
